In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
all_matches = []

In [3]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/b74092de/Ipswich-Town-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/33c895d4/Southampton-Stats']

In [5]:
from io import StringIO
import time

In [6]:
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    team_data = requests.get(team_url)
    try:
      matches = pd.read_html(StringIO(str(team_data.text)), match="Scores & Fixtures")[0]
      matches = matches[matches["Comp"] == "Premier League"]
    except ValueError:
        continue
    soup = BeautifulSoup(team_data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    sh_link = links[0].split('/')[:-3] + ['c9', 'shooting', f'{team_name}-Match-Logs-Premier-League']
    shooting_data = requests.get(f"https://fbref.com{'/'.join(sh_link)}")
    pas_link = links[0].split('/')[:-3] + ['c9', 'passing', f'{team_name}-Match-Logs-Premier-League']
    passing_data = requests.get(f"https://fbref.com{'/'.join(pas_link)}")
    def_link = links[0].split('/')[:-3] + ['c9', 'defense', f'{team_name}-Match-Logs-Premier-League']
    defense_data = requests.get(f"https://fbref.com{'/'.join(def_link)}")
    poss_link = links[0].split('/')[:-3] + ['c9', 'possession', f'{team_name}-Match-Logs-Premier-League']
    possession_data = requests.get(f"https://fbref.com{'/'.join(poss_link)}")
    try:
        shooting = pd.read_html(StringIO(str(shooting_data.text)), match="Shooting")[0]
    except ValueError:
        continue
    try:
        passing = pd.read_html(StringIO(str(passing_data.text)), match="Passing")[0]
    except ValueError:
        continue
    try:
        defense = pd.read_html(StringIO(str(defense_data.text)), match="Defensive Actions")[0]
    except ValueError:
        continue
    try:
        possession = pd.read_html(StringIO(str(possession_data.text)), match="Possession")[0]
    except ValueError:
        continue
    shooting.columns = shooting.columns.droplevel()
    passing.columns = passing.columns.droplevel()
    defense.columns = defense.columns.droplevel()
    possession.columns = possession.columns.droplevel()
    try:
        shooting_data = shooting[["Date",'Round', "Sh", "SoT", "Dist", "SoT%", "G/Sh", "G/SoT", "npxG", "npxG/Sh", "G-xG", "np:G-xG"]]
        passing_data = passing[["Date",'Round', "Cmp", "Att", "Cmp%", "TotDist", "PrgDist", "Ast", "xAG", "xA", "KP"]]
        defense_data = defense[["Date",'Round',"Tkl", "TklW","Def 3rd","Mid 3rd", "Att 3rd", "Blocks", "Sh", "Pass", "Int", "Tkl+Int", "Clr", "Err"]]
        possession_data = possession[["Date",'Round', "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Att Pen", "Live"]]

        # matches.fillna(-1)
        # shooting_data = shooting_data.dropna()
        # passing_data = passing_data.dropna()
        # defense_data =  defense_data.dropna()
        # possession_data = possession_data.dropna()
        data_frames = [shooting_data, passing_data, defense_data, possession_data]
        matches_all = matches.copy()
        for i in range(4):
            teams_data = matches_all.merge(data_frames[i].dropna(), on=["Date", "Round"], how='left')
        teams_data

    except ValueError:
        continue

    teams_data["Season"] = "2024-2025"
    teams_data["Team"] = team_name
    all_matches.append(teams_data)
    time.sleep(60)

In [7]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,notes,touches,def pen,def 3rd,mid 3rd,att 3rd,att pen,live,season,team
0,2024-08-16,20:00,Premier League,Matchweek 1,Fri,Away,L,0,1,Manchester Utd,...,NaN,540.0,75.0,223.0,212.0,117.0,21.0,540.0,2024-2025,Fulham
1,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,2,1,Leicester City,...,NaN,689.0,51.0,153.0,327.0,213.0,30.0,689.0,2024-2025,Fulham
2,2024-08-31,15:00,Premier League,Matchweek 3,Sat,Away,D,1,1,Ipswich Town,...,NaN,545.0,60.0,198.0,232.0,118.0,19.0,545.0,2024-2025,Fulham
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,D,1,1,West Ham,...,NaN,592.0,77.0,197.0,232.0,172.0,29.0,592.0,2024-2025,Fulham
4,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3,1,Newcastle Utd,...,NaN,484.0,58.0,149.0,170.0,169.0,37.0,484.0,2024-2025,Fulham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2025-04-26,15:00,Premier League,Matchweek 34,Sat,Home,NaN,NaN,NaN,Fulham,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-2025,Southampton
34,2025-05-03,15:00,Premier League,Matchweek 35,Sat,Away,NaN,NaN,NaN,Leicester City,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-2025,Southampton
35,2025-05-10,15:00,Premier League,Matchweek 36,Sat,Home,NaN,NaN,NaN,Manchester City,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-2025,Southampton
36,2025-05-18,15:00,Premier League,Matchweek 37,Sun,Away,NaN,NaN,NaN,Everton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-2025,Southampton


In [15]:
match_df.to_csv("matches_24_25.csv")